In [1]:
#Importing needed packages
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import pandas as pd
import B_funktioner
import pydst as pydst
dst = pydst.Dst(lang='en')



In [17]:
#Collection of datasets and cleaning
#This first section collects datasets from DST and cleans in order to get the wanted variables
#Tænker at dette skal gøres mere udførligt osv. Det er bare lige for vores eget overbliks skyld
#Måske skal vi overveje at have helt i starten noget der undersøger hvilke datasæt vi vil hente ala nedenstående

dst.get_subjects()

dst.get_tables(subjects=['02']).head(10)
dst.get_variables(table_id = 'FOLK1A')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'RAS200')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'UHELDK7')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'FOLK1E')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id = 'STRAF41')

dst.get_tables(subjects=['14']).head(10)
dst.get_variables(table_id='INDKP107')




,id,text,elimination,time,map,values
0,OMRÅDE,region,True,False,denmark_municipality_07,"[{'id': '000', 'text': 'All Denmark'}, {'id': ..."
1,ALDER,age,True,False,NaN,"[{'id': 'TOT', 'text': 'Age, total'}, {'id': '..."
2,KØN,sex,True,False,NaN,"[{'id': 'TOT', 'text': 'Total'}, {'id': '1', '..."
3,Tid,time,False,True,NaN,"[{'id': '2007', 'text': '2007'}, {'id': '2008'..."


In [35]:
#downloading the population data of the danish municipalities

variables = {'OMRÅDE':['*'],'Tid':['*']}
pop = dst.get_data(table_id = 'FOLK1A', variables=variables)

pop = B_funktioner.initial_rename(pop,'population')
pop = B_funktioner.only_keep_municipalities_and_years(pop,0,2008,2018)
pop = B_funktioner.sort_reset(pop)

del pop['KØN']
del pop['ALDER']
del pop['CIVILSTAND']
pop


,municipality,year,population
0,Aabenraa,2008,60189
1,Aabenraa,2009,60392
2,Aabenraa,2010,59978
3,Aabenraa,2011,59795
4,Aabenraa,2012,59600
...,...,...,...
1084,Ærø,2014,6393
1085,Ærø,2015,6276
1086,Ærø,2016,6290
1087,Ærø,2017,6177


In [5]:
#downloading the employment data of the danish municipalities

variables={'OMRÅDE':['*'], 'BEREGNING':['*'], 'TID':['*']}
empl = dst.get_data(table_id = 'RAS200', variables=variables)

empl = B_funktioner.initial_rename(empl,'employment rate')
empl = B_funktioner.only_keep_municipalities_and_years(empl,0,2008,2018)
#empl = B_funktioner.row_chooser(empl,BEREGNING,'Employment rate')

for val in ['Employment rate']: 
    I = empl.BEREGNING.str.contains(val)
    empl = empl.loc[I == True] # keep everything else

empl= B_funktioner.sort_reset(empl)

empl


,municipality,BEREGNING,year,HERKOMST,ALDER,KØN,employment rate
0,Aabenraa,Employment rate,2008,Total,"Age, total",Total,74.6
1,Aabenraa,Employment rate,2009,Total,"Age, total",Total,71.6
2,Aabenraa,Employment rate,2010,Total,"Age, total",Total,70.9
3,Aabenraa,Employment rate,2011,Total,"Age, total",Total,70.2
4,Aabenraa,Employment rate,2012,Total,"Age, total",Total,69.5
...,...,...,...,...,...,...,...
1084,Ærø,Employment rate,2014,Total,"Age, total",Total,65.1
1085,Ærø,Employment rate,2015,Total,"Age, total",Total,65.8
1086,Ærø,Employment rate,2016,Total,"Age, total",Total,67.0
1087,Ærø,Employment rate,2017,Total,"Age, total",Total,68.1


In [15]:
#UHELD - af Henrik Kyndal
#Downloading accident data 

variables={'OMRÅDE':['*'], 'UHELDA':['*'], 'TID':['*']}
accidents = dst.get_data(table_id = 'UHELDK7', variables=variables)

accidents = B_funktioner.initial_rename(accidents,'Number of accidents')
accidents = B_funktioner.only_keep_municipalities_and_years(accidents,0,2008,2018)
#accidents = B_funktioner.row_chooser(accidents,UHELDA,'All accidents')

I = accidents.UHELDA.str.contains('All accidents')
accidents = accidents.loc[I == True]

accidents= B_funktioner.sort_reset(accidents)
accidents



,municipality,UHELDA,year,BYLAND,UHELDSIT,Number of accidents
0,Aabenraa,All accidents,2008,Inside urban area,One-vehicle accidents,4
1,Aabenraa,All accidents,2009,Inside urban area,One-vehicle accidents,4
2,Aabenraa,All accidents,2010,Inside urban area,One-vehicle accidents,1
3,Aabenraa,All accidents,2011,Inside urban area,One-vehicle accidents,4
4,Aabenraa,All accidents,2012,Inside urban area,One-vehicle accidents,4
...,...,...,...,...,...,...
792,Ærø,All accidents,2009,Inside urban area,One-vehicle accidents,2
793,Ærø,All accidents,2011,Inside urban area,One-vehicle accidents,2
794,Ærø,All accidents,2015,Inside urban area,One-vehicle accidents,2
795,Ærø,All accidents,2016,Inside urban area,One-vehicle accidents,2


In [16]:
#Dowloading data on immigrants from non-western countries in the danish municipalities

variables={'OMRÅDE':['*'],'HERKOMST':{'*'}, 'TID':['*']}
immigrants = dst.get_data(table_id = 'FOLK1E', variables=variables)

immigrants = B_funktioner.initial_rename(immigrants,'Immigrants')
immigrants = B_funktioner.only_keep_municipalities_and_years(immigrants,0,2008,2018)
#immigrants = B_funktioner.row_chooser(immigrants,HERKOMST,'Immigrants from non-western countries')

I = immigrants.HERKOMST.str.contains('Immigrants from non-western countries')
immigrants = immigrants.loc[I == True]

immigrants = B_funktioner.sort_reset(immigrants)
immigrants



,municipality,HERKOMST,year,KØN,ALDER,Immigrants
0,Aabenraa,Immigrants from non-western countries,2008,Total,Total,1787
1,Aabenraa,Immigrants from non-western countries,2009,Total,Total,1822
2,Aabenraa,Immigrants from non-western countries,2010,Total,Total,1817
3,Aabenraa,Immigrants from non-western countries,2011,Total,Total,1887
4,Aabenraa,Immigrants from non-western countries,2012,Total,Total,1906
...,...,...,...,...,...,...
1084,Ærø,Immigrants from non-western countries,2014,Total,Total,104
1085,Ærø,Immigrants from non-western countries,2015,Total,Total,103
1086,Ærø,Immigrants from non-western countries,2016,Total,Total,123
1087,Ærø,Immigrants from non-western countries,2017,Total,Total,145


In [20]:
#Dowloading data on sentences of the danish municipalities

variables={'OMRÅDE':['*'], 'TID':['*']}
sentences = dst.get_data(table_id = 'STRAF41', variables=variables)

sentences = B_funktioner.initial_rename(sentences,'Number of sentences')
sentences = B_funktioner.only_keep_municipalities_and_years(sentences,0,2008,2018)
sentences = B_funktioner.sort_reset(sentences)

sentences


,municipality,year,ALDER,KØN,Number of sentences
0,Aabenraa,2008,"Age, total",Total,49197
1,Aabenraa,2009,"Age, total",Total,49132
2,Aabenraa,2010,"Age, total",Total,49145
3,Aabenraa,2011,"Age, total",Total,49146
4,Aabenraa,2012,"Age, total",Total,48994
...,...,...,...,...,...
1084,Ærø,2014,"Age, total",Total,5612
1085,Ærø,2015,"Age, total",Total,5561
1086,Ærø,2016,"Age, total",Total,5553
1087,Ærø,2017,"Age, total",Total,5489


In [29]:
#Dowloading data on average disposable income of the danish municipalities

variables={'OMRÅDE':['*'], 'TID':['*'], 'ENHED':['*']}
avginc = dst.get_data(table_id = 'INDKP107', variables=variables)

avginc = B_funktioner.initial_rename(avginc,'Avg disp income')
avginc = B_funktioner.only_keep_municipalities_and_years(avginc,0,2008,2018)

I = avginc.ENHED.str.contains('all people')
avginc = avginc.loc[I == True]

avginc = B_funktioner.sort_reset(avginc)

avginc


,municipality,year,ENHED,KOEN,UDDNIV,INDKOMSTTYPE,Avg disp income
0,Aabenraa,2008,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),125064
1,Aabenraa,2009,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),123277
2,Aabenraa,2010,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),130019
3,Aabenraa,2011,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),132330
4,Aabenraa,2012,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),133960
...,...,...,...,...,...,...,...
1064,Ærø,2014,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),132795
1065,Ærø,2015,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),129903
1066,Ærø,2016,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),136753
1067,Ærø,2017,Average income for all people (DKK),"Men and women, total",10 BASIC SCHOOL 8-10 grade,1 Disposable income (2+30-31-32-35),142416


In [36]:
#Merge of all the downloaded datasets

Tabel1 = pop

Tabel1 = Tabel1.join([empl['employment rate'],accidents['Number of accidents'],immigrants['Immigrants'],sentences['Number of sentences'],avginc['Avg disp income']])

Tabel1




,municipality,year,population,employment rate,Number of accidents,Immigrants,Number of sentences,Avg disp income
0,Aabenraa,2008,60189,74.6,4.0,1787,49197,125064.0
1,Aabenraa,2009,60392,71.6,4.0,1822,49132,123277.0
2,Aabenraa,2010,59978,70.9,1.0,1817,49145,130019.0
3,Aabenraa,2011,59795,70.2,4.0,1887,49146,132330.0
4,Aabenraa,2012,59600,69.5,4.0,1906,48994,133960.0
...,...,...,...,...,...,...,...,...
1084,Ærø,2014,6393,65.1,NaN,104,5612,NaN
1085,Ærø,2015,6276,65.8,NaN,103,5561,NaN
1086,Ærø,2016,6290,67.0,NaN,123,5553,NaN
1087,Ærø,2017,6177,68.1,NaN,145,5489,NaN
